In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
addr_list = ['Metro Manila', 'Luzon', 'Visayas', 'Mindanao']
SLA = pd.DataFrame([[3, 5, 7, 7], [5, 5, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7]], 
                   index = addr_list,
                   columns = addr_list)
SLA

,Metro Manila,Luzon,Visayas,Mindanao
Metro Manila,3,5,7,7
Luzon,5,5,7,7
Visayas,7,7,7,7
Mindanao,7,7,7,7


In [20]:
import datetime
holiday = [datetime.datetime.strptime('2020-03-01', "%Y-%m-%d"),
           datetime.datetime.strptime('2020-03-08', "%Y-%m-%d"),
           datetime.datetime.strptime('2020-03-15', "%Y-%m-%d"),
           datetime.datetime.strptime('2020-03-22', "%Y-%m-%d"),
           datetime.datetime.strptime('2020-03-25', "%Y-%m-%d"),
           datetime.datetime.strptime('2020-03-29', "%Y-%m-%d"),
           datetime.datetime.strptime('2020-03-30', "%Y-%m-%d"),
           datetime.datetime.strptime('2020-03-31', "%Y-%m-%d"),]
holiday

[datetime.datetime(2020, 3, 1, 0, 0),
 datetime.datetime(2020, 3, 8, 0, 0),
 datetime.datetime(2020, 3, 15, 0, 0),
 datetime.datetime(2020, 3, 22, 0, 0),
 datetime.datetime(2020, 3, 25, 0, 0),
 datetime.datetime(2020, 3, 29, 0, 0),
 datetime.datetime(2020, 3, 30, 0, 0),
 datetime.datetime(2020, 3, 31, 0, 0)]

In [8]:
dodf = pd.read_csv('delivery_orders_march_shortaddr.csv', index_col=0)

C:\Users\pb580\anaconda3\envs\ML_Env\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
dodf.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,Metro Manila,Metro Manila
1,2219624609,1583309968,1.583463e+09,1.583799e+09,Metro Manila,Metro Manila
2,2220979489,1583306434,1.583460e+09,NaN,Metro Manila,Metro Manila
3,2221066352,1583419016,1.583556e+09,NaN,Metro Manila,Metro Manila
4,2222478803,1583318305,1.583480e+09,NaN,Luzon,Metro Manila


In [11]:
temp = dodf.copy()

In [14]:
temp['pick'] = temp.pick.apply(lambda x: pd.to_datetime(x, unit='s'))
temp.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 08:39:57,1.583385e+09,NaN,Metro Manila,Metro Manila
1,2219624609,2020-03-04 08:19:28,1.583463e+09,1.583799e+09,Metro Manila,Metro Manila
2,2220979489,2020-03-04 07:20:34,1.583460e+09,NaN,Metro Manila,Metro Manila
3,2221066352,2020-03-05 14:36:56,1.583556e+09,NaN,Metro Manila,Metro Manila
4,2222478803,2020-03-04 10:38:25,1.583480e+09,NaN,Luzon,Metro Manila


In [15]:
attemp1 = temp['1st_deliver_attempt']
attemp2 = temp['2nd_deliver_attempt']

In [16]:
attemp1 = pd.to_datetime(attemp1, unit='s')
attemp2 = pd.to_datetime(attemp2, unit='s')

In [18]:
temp['1st_deliver_attempt'] = attemp1
temp['2nd_deliver_attempt'] = attemp2
temp.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,Metro Manila,Metro Manila
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,Metro Manila,Metro Manila
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,Metro Manila,Metro Manila
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,Metro Manila,Metro Manila
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,Luzon,Metro Manila


In [21]:
temp.to_csv('logistic.csv', index=False)

In [22]:
def least_days(buy, sell):
    return SLA.loc[buy, sell]

def is_late(pick, first, second, buy, sell):
    if second == np.nan:
        count = 0
        for hol in holiday:
            if pick <= hol and hol <= first:
                count+=1
        if first.day - pick.day - count <= least_days(buy, sell):
            return 0
        else:
            return 1
    else:
        if second.day - first.day > 3:
            return 1
        else:
            count = 0
            for hol in holiday:
                if pick <= hol and hol <= second:
                    count+=1
            if second.day - pick.day - count <= least_days(buy, sell):
                return 0
            else:
                return 1

In [37]:
pick_arr = temp['pick']
attemp1 = temp['1st_deliver_attempt']
attemp2 = temp['2nd_deliver_attempt']
buyeraddr = temp['buyeraddress']
selleraddr = temp['selleraddress']

In [47]:
is_late_arr = [is_late(pick_arr[i], attemp1[i], attemp2[i], buyeraddr[i], selleraddr[i]) for i in range(pick_arr.)]
is_late_arr

TypeError: 'list' object is not callable

In [29]:
temp['date_diff_0'] = attemp1 - pick_arr
temp['date_diff_1'] = attemp2 - attemp1
temp.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,date_diff_0,date_diff_1
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,Metro Manila,Metro Manila,2 days 20:29:21,NaT
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,Metro Manila,Metro Manila,1 days 18:34:28,3 days 21:09:00
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,Metro Manila,Metro Manila,1 days 18:35:45,NaT
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,Metro Manila,Metro Manila,1 days 14:08:45,NaT
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,Luzon,Metro Manila,1 days 21:03:15,NaT


In [30]:
order_ids = []
is_late = []

In [44]:
type(temp.pick[1])

pandas._libs.tslibs.timestamps.Timestamp